#### Import library

In [ ]:
import pandas as pd

#### Import Dataset

In [ ]:
try:
  monthly_expense_df = pd.read_excel('Monthly expenses.xlsx')
except FileNotFoundError:
  print("Monthly expenses.xlsx not found.")

try:
  payroll_df = pd.read_excel('Payroll.xlsx')
except FileNotFoundError:
  print("Payroll.xlsx not found.")

try:
  customer_lifespan_df = pd.read_excel('customer_lifespan_data.xlsx')
except FileNotFoundError:
  print("customer_lifespan_data.xlsx not found.")

try:
  daily_marketing_df = pd.read_excel('daily_marketing_spendings.xlsx')
except FileNotFoundError:
  print("daily_marketing.xlsx not found.")

try:
  receipts_history_df = pd.read_excel('receipts_history.xlsx')
except FileNotFoundError:
  print("receipts_history.xlsx not found.")


In [ ]:
monthly_expense_df.head()

,#,month,category,item,amount
0,1,2023-01-01,Server Costs,AWS Hosting,8000
1,2,2023-01-01,Server Costs,Google Cloud Storage,4000
2,3,2023-01-01,Software Licenses,Atlassian Jira,1200
3,4,2023-01-01,Software Licenses,Slack,800
4,5,2023-01-01,Software Licenses,Salesforce,1500


In [ ]:
payroll_df.head()

,month,department,employee_name,position,paid
0,2023-01-01,Sales,John Doe,Sales Manager,1500
1,2023-01-01,Sales,Jane Smith,Sales Associate,600
2,2023-01-01,Sales,Jim Brown,Sales Associate,700
3,2023-01-01,Sales,Laura Miller,Sales Associate,800
4,2023-01-01,Marketing,Alice Johnson,Marketing Manager,1650


In [ ]:
customer_lifespan_df.head()

,Unnamed: 0,start_date,churn_date
0,1000,2021-11-15,2022-09-14
1,1001,2022-04-15,2023-02-16
2,1002,2022-10-30,2023-02-04
3,1003,2021-08-22,2023-02-07
4,1004,2021-08-23,2022-02-02


In [ ]:
daily_marketing_df.head()

,date,channel,spending
0,2023-01-01,Google Ads,784
1,2023-01-01,Facebook Ads,659
2,2023-01-01,LinkedIn Ads,729
3,2023-01-01,Twitter Ads,292
4,2023-01-02,Google Ads,935


In [ ]:
receipts_history_df.head()

,date,customer_id,receipt_amount,new_customer
0,2023-01-01,2653,67,1
1,2023-01-01,2731,271,1
2,2023-01-01,1277,231,0
3,2023-01-01,2094,107,0
4,2023-01-01,1314,416,0


#### Calculate Unit Economics metrics

In [ ]:
# Calculate CAC (Customer Acquisition Cost)
try:
  cac = daily_marketing_df['spending'].sum() / len(customer_lifespan_df)
  print(f"CAC: {cac}")
except KeyError:
  print("Error: Marketing Spend or Customer ID columns not found in the respective dataframes.")


# Calculate ARPU (Average Revenue Per User)
try:
  arpu = receipts_history_df['receipt_amount'].sum() / len(customer_lifespan_df)
  print(f"ARPU: {arpu}")
except KeyError:
  print("Error: Revenue or Customer ID columns not found in the respective dataframes.")

# Calculate COGS (Cost of Goods Sold) - You'll need to identify relevant cost columns in your data
try:
  cogs = monthly_expense_df['amount'].sum()
  print(f"COGS: {cogs}")
except KeyError:
  print("Error: Cost of Goods column not found in Monthly expenses dataframe.")


# Calculate Gross Margin
try:
  gross_margin = (receipts_history_df['amount'].sum() - cogs) / receipts_history_df['amount'].sum()
  print(f"Gross Margin: {gross_margin}")
except KeyError:
  print("Error: Revenue or Cost of Goods columns not found in the respective dataframes.")

# # Calculate LTV (Lifetime Value)
# try:
#   ltv = arpu * customer_lifespan_df['Customer Lifespan (months)'].mean()
#   print(f"LTV: {ltv}")
# except KeyError:
#   print("Error: Customer Lifespan (months) column not found in the Customer Lifespan dataframe.")


# Calculate LTV / CAC
try:
  ltv_cac = ltv / cac
  print(f"LTV / CAC: {ltv_cac}")
except NameError:
  print("Error: CAC or LTV not calculated yet. Please ensure the relevant calculations are successful.")


CAC: 1994.78
ARPU: 2412.06
COGS: 90960
Error: Revenue or Cost of Goods columns not found in the respective dataframes.
Error: CAC or LTV not calculated yet. Please ensure the relevant calculations are successful.


In [ ]:
def get_last_month(dataset, date_col):
  last_month = dataset[date_col].max().month
  cond = dataset[date_col].dt.month == last_month
  return dataset[cond]

In [ ]:
get_last_month(receipts_history_df,'date')

,date,customer_id,receipt_amount,new_customer
618,2023-03-01,1062,103,0
619,2023-03-01,2243,157,0
620,2023-03-01,1166,372,0
621,2023-03-01,2406,426,1
622,2023-03-01,2761,41,1
...,...,...,...,...
924,2023-03-31,2079,358,0
925,2023-03-31,2089,375,0
926,2023-03-31,1355,249,0
927,2023-03-31,1062,111,0


In [ ]:
#Get the total of new_customer in 2023-03 in receipts_history
receipts = get_last_month(receipts_history_df,'date')
total_new_customers = receipts['new_customer'].sum()
#Total sales and Marketing expenses
monthly_expense_df_last_month = get_last_month(monthly_expense_df, 'month')
monthly_expense_df_salesforce = monthly_expense_df_last_month[monthly_expense_df_last_month['item'] == 'Salesforce']
monthly_expense = monthly_expense_df_salesforce['amount'].sum()

payroll_last_month = get_last_month(payroll_df, 'month')
payroll_sale_marketing = payroll_last_month[payroll_last_month['department'].isin(['Sales', 'Marketing'])]
payroll = payroll_sale_marketing['paid'].sum()

daily_marketing_last_month = get_last_month(daily_marketing_df, 'date')
daily_marketing = daily_marketing_last_month['spending'].sum()

In [ ]:
# Calculate CAC (Customer Acquisition Cost)
total_expenses = monthly_expense + payroll + daily_marketing
cac = total_expenses / total_new_customers
print(f"CAC: {cac}")

CAC: 1213.968253968254


In [ ]:
receipts_new_customers = receipts[receipts['new_customer'] == 1]
total_receipt_amount_new_customers = receipts_new_customers['receipt_amount'].sum()
print(f"Total receipt amount for new customers: {total_receipt_amount_new_customers}")


Total receipt amount for new customers: 17320


In [ ]:
# Calculate total revenue and number of users
receipts_new_customers = receipts[receipts['new_customer'] == 1]
total_receipt_amount_new_customers = receipts_new_customers['receipt_amount'].sum()
total_users = receipts['customer_id'].count()
# Calculate ARPU (Average Revenue Per User)
arpu = total_receipt_amount_new_customers / total_new_customers
print(f"ARPU: {arpu}")

ARPU: 274.92063492063494


In [ ]:
# Get last month's production cost
production = ['AWS Hosting', 'Google Cloud Storage', 'Atlassian Jira']
shared = ['Slack', 'Zoom']
production_cost = monthly_expense_df_last_month[monthly_expense_df_last_month['item'].isin(production)]['amount'].sum()
shared_cost = monthly_expense_df_last_month[monthly_expense_df_last_month['item'].isin(shared)]['amount'].sum()*0.6
print(f"Production cost: {production_cost}")
# Get last month's salary cost for the engineering department
salary_cost = payroll_last_month[payroll_last_month['department'] == 'Engineering']['paid'].sum()
print(salary_cost)
COGS = production_cost + salary_cost + shared_cost
print(f"COGS: {COGS}")

Production cost: 14200
5200
COGS: 20264.0


In [ ]:
total_revenue = receipts['receipt_amount'].sum()
print(f"Total revenue: {total_revenue}")
gross_margin = (total_revenue - COGS) / total_revenue *100
print(f"Gross Margin: {gross_margin}")

Total revenue: 83033
Gross Margin: 75.5952452639312


In [ ]:
customer_lifespan = (customer_lifespan_df['churn_date'] - customer_lifespan_df['start_date']).mean()
print(f"Customer lifespan: {customer_lifespan}")

Customer lifespan: 295 days 05:45:36


In [ ]:
lifespan_data = customer_lifespan_df.dropna(subset=['churn_date'])

In [ ]:
lifespan_data['lifespan_days'] = (lifespan_data['churn_date'] - lifespan_data['start_date']).dt.days

In [ ]:
avg_lifespan_days = lifespan_data['lifespan_days'].mean()
avg_lifespan_months = avg_lifespan_days / 30
print(f"Average customer lifespan (months): {avg_lifespan_months}")

Average customer lifespan (months): 9.841333333333333


In [ ]:
LTV = arpu * avg_lifespan_months * gross_margin/100
print(f"LTV: {LTV}")

LTV: 2045.2940765452022


In [ ]:
ltv_cac = LTV / cac
print(f"LTV / CAC: {ltv_cac}")

LTV / CAC: 1.6848002984093584


### Conclusion:

Based on the calculated unit economics, the company is performing reasonably well but faces areas that need attention:

1. **Customer Acquisition Cost (CAC)** is **1213.97**, which indicates how much the company spends on acquiring a new customer. This number is on the higher side and directly affects profitability.
   
2. **Average Revenue Per User (ARPU)** is **274.92**, reflecting the revenue generated per customer. While this provides a steady stream of income, the relatively high CAC suggests a need to either reduce acquisition costs or increase ARPU to balance out the expenses.
   
3. **Cost of Goods Sold (COGS)** is **20264.0**, which includes both production and salary costs. This figure significantly affects the company's profitability, and controlling these costs will be crucial.
   
4. **Gross Margin** is **75.60%**, which is healthy and shows that the company retains a substantial portion of its revenue after accounting for direct costs. This margin indicates operational efficiency and a scalable business model.
   
5. **Customer Lifetime Value (LTV)** is **2045.29**, showing the total revenue a company can expect from a customer over their lifetime. While this number is positive, it can be improved by increasing ARPU or retaining customers for longer periods.
   
6. **LTV/CAC Ratio** is **1.68**, which is decent but should ideally be higher. A ratio above 3 is considered ideal for sustainable growth, meaning the company earns three times more from a customer than it spends to acquire them. At the current ratio, customer acquisition is close to breaking even but not optimally profitable.

### Recommendations:

1. **Reduce CAC**: The CAC of **1213.97** is relatively high. Efforts should be made to lower this by optimizing marketing strategies, focusing on organic growth channels, or improving customer targeting to acquire users at a lower cost. Exploring partnerships and referral programs could also help reduce acquisition expenses.

2. **Increase ARPU**: To balance the high CAC, the company should explore ways to increase the ARPU of **274.92**. This could be achieved through upselling or cross-selling additional products or services, increasing pricing for premium services, or offering tailored bundles that encourage higher spending.

3. **Improve LTV/CAC Ratio**: With an LTV/CAC ratio of **1.68**, the company should aim to push this closer to or above 3. Improving customer retention strategies, such as offering better customer support, creating loyalty programs, or enhancing product features, can increase the LTV.

4. **Optimize COGS**: Since the **COGS** is significant at **20264.0**, efforts to reduce operating costs—without sacrificing quality—would improve the company’s bottom line. Investigating more cost-efficient suppliers, renegotiating contracts, or automating parts of the production process could help reduce costs.

5. **Enhance Gross Margin**: While the gross margin of **75.60%** is solid, continuing to streamline operational processes or scaling the business could help maintain or even improve this margin as the company grows.

By focusing on these areas, the company can strengthen its profitability and long-term sustainability, positioning itself for better financial health and growth.
